# Deep Interactive Evolution (for music)

This is an unofficial implementation of the algorithm found in the paper "Deep Interactive Evolution" (https://arxiv.org/abs/1801.08230). In contrast to the original paper, this implementation uses a VAE (instead of a GAN) trained on 2-bar music (instead of images), to evolve a 2-bar sample the user prefers. 

# Setup
To demonstrate the generality of the algorithm to any latent variable model, we use a pretrained VAE model by Google Magenta, MusicVAE (found here: https://colab.research.google.com/notebook#fileId=/v2/external/notebooks/magenta/music_vae/music_vae.ipynb).

Installs and imports all packages for the Magenta MusicVAE, alongside loading the basic 2-bar loop model (a bidirectional LSTM encoder based VAE).

In [19]:
import glob

print'Copying checkpoints and example MIDI from GCS. This may take a few minutes...'
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab/* /content/

print 'Installing dependencies...'
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

if glob.glob('/content/magenta*.whl'):
  !pip install -qU /content/magenta*.whl
else:
  !pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib.
import ctypes.util
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return ctypes.util.find_library(lib)

ctypes.util.find_library = proxy_find_library


print 'Importing libraries and defining some helper functions...'
from google.colab import files
import magenta.music as mm
from magenta.music.sequences_lib import concatenate_sequences
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow as tf


def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)
  
def slerp(p0, p1, t):
  """Spherical linear interpolation."""
  omega = np.arccos(np.dot(np.squeeze(p0/np.linalg.norm(p0)), np.squeeze(p1/np.linalg.norm(p1))))
  so = np.sin(omega)
  return np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5, 
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  _, mu, _ = model.encode([start_seq, end_seq], assert_same_length)
  z = np.array([slerp(mu[0], mu[1], t) for t in np.linspace(0, 1, num_steps)])
  note_sequences = model.decode(
      length=max_length,
      z=z,
      temperature=temperature)

  print 'Start Seq Reconstruction'
  play(note_sequences[0])
  print 'End Seq Reconstruction'
  play(note_sequences[-1])
  print 'Mean Sequence'
  play(note_sequences[num_steps // 2])
  print 'Start -> End Interpolation'
  interp_seq = concatenate_sequences(note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print 'Done'

# Load the pre-trained model.
mel_2bar_config = configs.CONFIG_MAP['cat-mel_2bar_big']
mel_2bar = TrainedModel(mel_2bar_config, batch_size=4, checkpoint_dir_or_path='/content/checkpoints/mel_2bar_big.ckpt')

Copying checkpoints and example MIDI from GCS. This may take a few minutes...
Installing dependencies...
Importing libraries and defining some helper functions...
Done


In [56]:
rangek = 10
z_length = 512
p = 0.5
foreign = 2
model = mel2bar
np.random.seed(0)

z = np.random.randn(rangek, z_length)
m, n = rangek, z_length

print 'Evolution hyperparamter initialization and model definiton complete'

Evolution hyperparamter initialization complete


Some helper functions for the evolution process, as outlined in algorithm 2 of the paper


In [0]:
def uniform(population):
    pop_len = len(population)
    a = population[np.random.randint(0, pop_len)]
    b = population[np.random.randint(0, pop_len)]
    mask = np.random.binomial(1, 0.5, z_length)
    return mask*a + (1-mask)*b

def mutate(individual, std):
    mutate_cond = np.random.binomial(1, p, 1)
    noise = std*np.random.randn(1, z_length)
    return individual + mutate_cond * noise

def evolve(z, indices, mutate_rate, shuffle=True):
    selections = z[indices]
    mutate_var = mutate_rate

    diff = m - len(selections)
    x = np.max([0, diff])
    cross = np.array([mutate(uniform(selections), mutate_var) for i in range(x - foreign)]).squeeze(axis=1)

    x = np.min((foreign, diff))
    new = np.random.randn(x, z_length)

    selections = np.array([mutate(selection, mutate_var) for selection in selections]).squeeze(axis=1)

    z = np.vstack((selections, cross, new))
    
    # if not shuffle, the first n(selected) samples are mutated selected samples, 
    # the last n(foreign) samples are foreign samples, and all samples inbetween are crossovers
    if shuffle:
        z = np.random.shuffle(z) 
    
    return z
    

#### Some of the samples I obtained:

Start: https://github.com/irhumshafkat/deepIEmidi/blob/master/start.wav

8 generations later: https://github.com/irhumshafkat/deepIEmidi/blob/master/8thgen.wav

Colaboratory does not have interactive features, so to simulate the user selection process, keep running the next two cells in sequence.

In [70]:
note_sequences = model.decode(
      length=32,
      z=z,
      temperature=0.8)

print 'Start Seq Reconstruction'
for i in range(rangek):
  print i 
  play(note_sequences[i])

Start Seq Reconstruction
0


1


2


3


4


5


6


7


8


9


If at any stage you'd like to seperately store a sample in memory, run the next cell with the index of the desired sample. Else, do not run, skip. 

Replace the elements in list selected with the actual indices of the samples you wish to keep and run the next cell, after which, re-run the previous cell.

In [0]:
best_yet = z[0] #insert the index of the best so far
play(best_yet)

In [0]:
list_selected = [1, 6]
z = evolve(z, list_selected, 0.2, shuffle=True)